# Hypothèse 1 : La surface utile est proportionnelle à la surface au sol

La fonction calculate_surface_utile du module 13_potentiel_solaire/algorithme/potentiel_solaire/features/solar_potential.py convertit la surface au sol en surface utile comme suit:

### Code original de référence :
```python
def calculate_surface_utile(surface_totale_au_sol: float):
    """Calcule la surface utile pour le PV.
    Pour le moment il s agit d'un simple ratio.
    @TODO Remplacer par une formule plus fine
    :param surface_totale_au_sol: surface totale au sol du batiment
    :return: la surface utile pour installation de panneaux PV
    """
    if surface_totale_au_sol <= 100:
        return 0
    if 100 < surface_totale_au_sol < 500:
        ratio = 0.4 * surface_totale_au_sol / 5000 + 0.2
        return ratio * surface_totale_au_sol
    return 0.6 * surface_totale_au_sol
```


 - Si la surface au sol est inférieure à 100 m², alors la surface utile est nulle
 - Si la surface au sol est comprise entre 100 et 500m² alors la surface utile est $$Surface\_utile = 8 \times 10^{-5} \cdot S^2 + 0.2 \cdot S$$
 - Si la surface au sol est supérieure à 500m², $$Surface\_utile = 0.6 \cdot S$$
où $S$ = surface totale au sol

Je vois un premier problème - si la surface au sol est comprise entre 100 et 500m², la fonction est quadratique et n'est pas linéaire. Je ne sais pas si c'est ce qui est attendu.


## Récupération des résultats des deux methodes

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from potentiel_solaire.constants import RESULTS_FOLDER, DATA_FOLDER
from potentiel_solaire.sources.utils import download_file

results_with_simplified_method = gpd.read_file(RESULTS_FOLDER / "priotirized_schools_buildings.gpkg", layer="results_with_simplified_method")
results_with_segmentation = pd.read_csv(RESULTS_FOLDER / "roof_segments_potential.csv").rename(columns={"surface": "roof_surface"})

# Calcule de la surface de toit plat
results_with_segmentation["flat_roof_surface"] = results_with_segmentation["roof_surface"].where(
    results_with_segmentation["slope_bin_min"] == 0, 0
)

# Calcul de la surface utile (methode par segmentation) en utilisant la meme definition que IDF
# Surface présentant une irradiation suffisante > 900 kWh/m².an et n’ayant pas d’obstacle (cheminée, velux, aération…)
results_with_segmentation["utility_surface"] = results_with_segmentation["roof_surface"].where(
    results_with_segmentation["solar_irradiation"] > 900, 0
)

# Calcul du potentiel solaire en ne prenant que les segments avec surface utile > 0
results_with_segmentation["solar_potential"] = results_with_segmentation["solar_potential"].where(
    results_with_segmentation["utility_surface"] > 0, 0
)

results_with_segmentation["slope_bin_label"] = results_with_segmentation["slope_bin_min"].astype(str) + "-" + results_with_segmentation["slope_bin_max"].astype(str)



# Synthese des resultats de la methode avec segmentation au niveau des batiments
results_with_segmented_roofs = results_with_segmentation.groupby(by="cleabs_bat").agg(
    roof_surface=("roof_surface", "sum"),  # methode avec segmentation (MNS)
    flat_roof_surface=("flat_roof_surface", "sum"),  # methode avec segmentation (MNS)
    utility_surface=("utility_surface", "sum"),  # methode avec segmentation (MNS)
    solar_potential=("solar_potential", "sum"),  # methode avec segmentation (MNS)
    latitude=("latitude", "first"),  # Ajout latitude
    longitude=("longitude", "first"),  # Ajout longitude
    slope_bin_label=("slope_bin_label", "first"),  # Ajout longitude
).reset_index()

# Comparaison des resultats entre les deux methodes
results_comparison_buildings = pd.merge(
    results_with_simplified_method[["code_region", "code_departement", "identifiant_de_l_etablissement", "cleabs_bat", "surface_totale_au_sol", "surface_utile", "potentiel_solaire"]],
    results_with_segmented_roofs[["cleabs_bat", "roof_surface", "flat_roof_surface", "utility_surface", "solar_potential", "longitude", "latitude", "slope_bin_label"]],
    on="cleabs_bat",
)

# Synthese des resultats des deux methodes a l echelle des etablissements
results_comparison_schools = results_comparison_buildings.groupby("identifiant_de_l_etablissement").agg(
    code_region=("code_region", "first"),  # Prendre la première valeur (devrait être la même pour tous)
    code_departement=("code_departement", "first"),  # Idem pour le département
    surface_totale_au_sol=("surface_totale_au_sol", "sum"),  # methode simplifiée
    surface_utile=("surface_utile", "sum"),  # methode simplifiée
    potentiel_solaire=("potentiel_solaire", "sum"),  # methode simplifiée
    roof_surface=("roof_surface", "sum"),  # methode avec segmentation (MNS)
    flat_roof_surface=("flat_roof_surface", "sum"),  # methode avec segmentation (MNS)
    utility_surface=("utility_surface", "sum"),  # methode avec segmentation (MNS)
    solar_potential=("solar_potential", "sum"),  # methode avec segmentation (MNS)
    latitude=("latitude", "first"),  # Ajout latitude
    longitude=("longitude", "first"),  # Ajout longitude
    slope_bin_label=("slope_bin_label",lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]),  # Ajout bin de pente
).reset_index()

In [ ]:
def plot_scatter(
    x: pd.Series,
    y: pd.Series,
    title: str, 
    xlabel: str,
    ylabel: str,
    xlim: tuple = None,
    ylim: tuple = None,
    identity_line: bool = True
):
    """Plot the results of two series against each other.
    
    Args:
        x (pd.Series): The x-axis data.
        y (pd.Series): The y-axis data.
        title (str): The title of the plot.
        xlabel (str): The label for the x-axis.
        ylabel (str): The label for the y-axis.
        xlim (tuple, optional): The limits for the x-axis. Defaults to None.
        ylim (tuple, optional): The limits for the y-axis. Defaults to None.
        identity_line (bool, optional): Whether to plot the identity line. Defaults to True.
    """
    plt.rcParams['figure.figsize'] = [20, 10]

    if identity_line:
        # plot identity line
        identity_line = [0, x.max()]
        plt.plot(identity_line, identity_line, "r--")

    
    # Compute and plot linear regression line
    model = LinearRegression()
    model.fit(x.values.reshape(-1, 1), y)
    y_pred = model.predict(x.values.reshape(-1, 1))
    plt.plot(x, y_pred, "g--")
    
    # Compute and display R^2 score and model parameters
    r2 = model.score(x.values.reshape(-1, 1), y)
    slope = model.coef_[0]
    intercept = model.intercept_

    # Display R², slope, and intercept on the plot
    plt.text(
        0.05, 0.95,
        f"R²: {r2:.3f}\nSlope: {slope:.3f}\nIntercept: {intercept:.3f}",
        transform=plt.gca().transAxes,
        fontsize=14,
        verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.7)
    )
    
    # compare x and y
    plt.scatter(x, y, alpha=0.8)

    # Set the title and labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    #  Set the limits for x and y axes if provided
    if xlim:
        plt.xlim(xlim)
    
    if ylim:
        plt.ylim(ylim)

    # Show the plot 
    plt.show()
    return [slope, intercept]

In [ ]:
slope, intercept = plot_scatter(
    x=results_comparison_schools["surface_totale_au_sol"],
    y=results_comparison_schools["utility_surface"],
    title="Relation entre la surface totale au sol et la surface utile réelle (obtenue par segmentation des toits)",
    xlabel="Surface totale au sol (m²)",
    ylabel="Surface utile au sol (m²)",
    identity_line=False,
)

In [ ]:
import numpy as np

# Calcul des valeurs prédites et des résidus
x = results_comparison_schools["surface_totale_au_sol"]
y = results_comparison_schools["utility_surface"]
y_pred = slope * x + intercept
residuals = y - y_pred

# Plot des résidus
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Résidus vs valeurs prédites
ax1.scatter(y_pred, residuals, alpha=0.6, s=30)
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.8)
ax1.set_xlabel('Valeurs prédites (m²)')
ax1.set_ylabel('Résidus (m²)')
ax1.set_title('Résidus vs Valeurs prédites')
ax1.grid(True, alpha=0.3)

# 2. Résidus vs variable x
ax2.scatter(x, residuals, alpha=0.6, s=30)
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.8)
ax2.set_xlabel('Surface totale au sol (m²)')
ax2.set_ylabel('Résidus (m²)')
ax2.set_title('Résidus vs Surface au sol')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistiques des résidus
print(f"Résidus - Moyenne: {residuals.mean():.2f}")
print(f"Résidus - Écart-type: {residuals.std():.2f}")
print(f"Résidus - Min: {residuals.min():.2f}")
print(f"Résidus - Max: {residuals.max():.2f}")

On voit que l'erreur faite sur la prédiction augmente avec la surface. 

### Conclusion

Au-delà d'une certaine surface utile, le potentiel solaire est systématiquement surestimé, tandis que le "solar potential" ou potentiel "réel" tend à être plus faible.
Je pense que le modèle linéaire pour convertir la surface au sol en surface utile ne peut pas tenir compte de toit obstrué par exemple, en particulier quand la surface est grande. 
Si l'on prend le bin 1 (surface utile de 2334m² à 4639m²) on voit que le potentiel solaire n'est jamais plus faible que 0.5 M de kWh.
On pourrait imaginer qu'un toit complètement ou partiellement obstrué aura un potentiel solaire plus faible - une simple régression linéaire entre surface au sol et surface utile ne tient pas compte de cette effet. Le potentiel solaire est surestimé.
Je pense qu'il est difficile de corriger ceci. 

Une correction via régression linéaire entre les deux méthodes d'estimation du potentiel solaire pourrait limiter les surestimations.

In [ ]:
# Hypothèse 2 : La pente des toits est nulle

In [ ]:
total_flat_surface = results_with_segmentation["flat_roof_surface"].sum()
total_roof_surface = results_with_segmentation["roof_surface"].sum()
ratio_flat_roof = total_flat_surface / total_roof_surface

print(f"Surface des toits plats: {total_flat_surface} m²")
print(f"Surface totale des toits: {total_roof_surface} m²")
print(f"Ratio des toits plats: {ratio_flat_roof:.2%}")

results_with_segmentation["slope_bin_label"] = results_with_segmentation["slope_bin_min"].astype(str) + "-" + results_with_segmentation["slope_bin_max"].astype(str)
h = results_with_segmentation.groupby(by=["slope_bin_min", "slope_bin_label"])["roof_surface"].sum().reset_index().sort_values(by="slope_bin_min")

plt.rcParams['figure.figsize'] = [20, 10]
plt.bar(h["slope_bin_label"], h["roof_surface"])
plt.title("Surface des toits par pente")
plt.xlabel("Pente des toits (°)")
plt.ylabel("Surface des toits (m²)")
plt.xticks(rotation=45)

plt.show()

Une majorité de toits à une pente inférieure à 5°. Même si la pente a un effet sur le potentiel solaire, il est probablement faible pour une grande partie des toits.

In [ ]:
def create_slope_histograms(df):
    """Histogrammes par bins de pente simplifiés: 0-5 et 5+ (échelle linéaire)"""
    
    # Créer les bins de pente simplifiés
    df_analysis = df.copy()
    df_analysis['slope_category'] = df_analysis['slope_bin_label'].apply(
        lambda x: '0-5°' if x.startswith('0-') else '5+°'
    )
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=['Pente 0-5°', 'Pente 5+°']
    )
    
    colors = ['#1f77b4', '#ff7f0e']  # Bleu, Orange
    
    # Échelle fixe de 0 à 1M
    min_val = 0
    max_val = 1000000  # 1M kWh
    bins = np.linspace(min_val, max_val, 100)
    bin_size = bins[1] - bins[0]
    
    categories = ['0-5°', '5+°']
    for i, category in enumerate(categories):
        col = i + 1
        data_cat = df_analysis[df_analysis['slope_category'] == category]
        
        if len(data_cat) == 0:
            print(f"Aucune donnée pour {category}")
            continue
        
        # Histogramme méthode simplifiée
        fig.add_trace(
            go.Histogram(
                x=data_cat['potentiel_solaire'],
                name='Méthode simplifiée' if i == 0 else None,
                opacity=0.7,
                xbins=dict(start=min_val, end=max_val, size=bin_size),
                marker_color=colors[0],
                showlegend=(i == 0)
            ),
            row=1, col=col
        )
        
        # Histogramme MNS
        fig.add_trace(
            go.Histogram(
                x=data_cat['solar_potential'],
                name='Méthode MNS' if i == 0 else None,
                opacity=0.7,
                xbins=dict(start=min_val, end=max_val, size=bin_size),
                marker_color=colors[1],
                showlegend=(i == 0)
            ),
            row=1, col=col
        )
        
        print(f"{category}: {len(data_cat)} établissements")
    
    fig.update_layout(
        title_text="Comparaison potentiel solaire par catégories de pente",
        height=500,
        barmode='overlay'
    )
    
    # Forcer l'échelle de 0 à 1M sur l'axe x
    fig.update_xaxes(title_text="Potentiel solaire (kWh)", range=[0, 1000000])
    fig.update_yaxes(title_text="Fréquence")
    
    return fig

fig_slopes = create_slope_histograms(results_comparison_schools)
fig_slopes.show()

### Conclusion

Le potentiel solaire semble davantage surestimé pour les batiments ayant une pente supérieure à 5°. L'erreur faite sur le potentiel solaire croit avec la pente.
Les distributions semblent relativement proches pour les pentes inférieures à 5°. L'effet est probablement négligeable pour la plupart des batiments.


## Résumé

Le biais entre les deux méthodes d'estimation du potentiel solaire semble provenir principalement de la conversion entre la surface au sol et la surface utile.
Une régression linéaire entre les résultats fournis par les deux méthodes permettrait de réduire la surestimation faite avec la méthode simplifiée de calcul du potentiel solaire. Méthode simplifiée est ici le cas où l'on suppose une relation linéaire entre surface au sol et surface utile et où l'on considère la pente des toits comme étant nulle.